# Lab 4 

## Database Design 

### 3 tables:
    1.) Gene table: contains tuples of:
            {geneID, species, sequence}
    2.) Pathway descriptions: contains tuples of:
            {pathway name, pathway description}
    3.) Enzyme: contains tuples of:
            {enzyme name, geneID (, function)}
            
## Genes Chosen:
### Glycolysis:
alcohol dehydrogenase \
hexokinase \
triosephosphate isomerase \
glucokinase \

### TCA:
phosphoenolpyruvate carboxykinase \
pyruvate carboxylase \
pyruvate synthase \
citrate synthase \
 
### Pentose Phosphate:
glucosaminate ammonia-lyase \
2-dehydro-3-deoxy-phosphogluconate \
gluconate dehydratase \
glucose oxidase \

# Handling:
If the gene is not found via an ENTREZ search (or if the ENTREZ search returns a nucleotide sequence that not accurate (i.e. is too long), "N/A" is inserted into the database

# Note to the Grader:
Hey Zoe! I really appreciated the comments you made on my Lab 2 notebook. If you have any thoughts at all, like no matter how mundane or trivial, I'd really like to hear them! (e.g. "This organization is kinda offputting"/"you could have made a function to do this a lot easier here")

In [1]:
!ls

2019_9_29_Lab4.ipynb		fixedLargeFileError.txt  mus musculus_leptin.gb
2019_9_29_Lab4_database.db.zip	mus musculus_leptin.fa


In [2]:
import sqlite3
from Bio import Entrez
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import generic_protein
from Bio import SeqIO
import time

In [3]:
# Set up database
conn = sqlite3.connect("2019_9_29_Lab4_database.db")
c = conn.cursor()

# c.execute("DROP TABLE genes")
# conn.commit()
# c.execute("DROP TABLE enzyme")
# conn.commit()
c.execute("CREATE TABLE genes (geneId TEXT, type TEXT, species TEXT, sequence TEXT)")
conn.commit()
c.execute("CREATE TABLE enzyme (geneId TEXT, protein TEXT)")
conn.commit()

In [4]:
# Testing Entrez:
Entrez.email = 'jerry.yang@berkeley.edu'

# Was recieving HTTPS Error: Too many requests
    # Made API key: https://ncbiinsights.ncbi.nlm.nih.gov/2017/11/02/new-api-keys-for-the-e-utilities/
Entrez.api_key = "6978aa54332d348f9d96988a2406a6b0ab08"

# handle = Entrez.esearch(db='nucleotide',
#                        term='homo sapiens[ORGN] leptin',
#                        sort='relevance',
#                        idtype='acc',
#                        retmax = 1)

In [5]:
# for entry in Entrez.read(handle)['IdList']:
#     result = Entrez.efetch(db='nucleotide', id=entry, rettype='fasta', retmode='text')
    # print(result.read())
    
    
#     for seq_rec in SeqIO.parse("seqs.fa", "fasta"):   
#     seqs[seq_rec.id] = seq_rec

In [6]:
# Creating a helper function to run Entrez
def search(term, species='homo sapiens'):
    """ Function to search Entrez for a particular gene, given species and 
            search term
            
        Returns the top hit for that search as a dictionary containing the following information:
            {   "organism": [organism], 
                "sequence": [seq],
                "type": [rna or dna],
                "geneId": [geneID]
                }
            
        E.g.
        > search(species = "mus musculus", 'leptin')
        
        
        ">NM_008493.3 Mus musculus leptin (Lep), mRNA
        GAGGGATCCCTGCTCCAGCAGCTGCAAGGTGCAAGAAGAAGAAGATCCCAGGGAGGAAAATGTGCTGGAG ..."
        
        """
    handle = Entrez.esearch(db='nucleotide',
                   term='{species}[ORGN] {searchTerm}'.format(species = species, searchTerm = term),
                   sort='relevance',
                   idtype='acc',
                   retmax = 1)
    for entry in Entrez.read(handle)['IdList']:
        result = Entrez.efetch(db='nucleotide', id=entry, rettype='gb')
        # print(result.read())
        
        for rec in SeqIO.parse(result, "gb"):
            # print(rec.annotations)
            addToEnzymeDB(term, rec.annotations.get("accessions")[0], "enzyme")
            def checkSeq(seq):
                # Checks the sequence to see it is not too large
                max_len = 10000
                return len(str(seq)) < max_len
            seq = ""
            if checkSeq(str(rec.seq)):
                seq = str(rec.seq)
            else:
                seq = "N/A" # Sequence return is too long (invalid)
                
            return {"organism": rec.annotations.get('organism'),
                    "geneID": rec.annotations.get("accessions")[0],
                    "type": rec.annotations.get("molecule_type"),
                    "sequence": seq #str(rec.seq)
                    }
#             print(rec.annotations.get('organism'))
#             print(rec.seq)
    
#     def genOutputFastaName(species, term):
#         """ Helper function to systematically generate an output name"""
#         return "{sp}_{term}.gb".replace(" ", "_").format(sp = species, term = term)
    return {"organism": "N/A",
                    "geneID": "N/A",
                    "type": "N/A",
                    "sequence": "N/A"
                    }
#     with open(genOutputFastaName(species, term), "w") as output_handle:
#         gene_data = result.read()
        # output_handle.write(gene_data)
        
        # print(gene_data)
# search('leptin', species = 'mus musculus')

In [7]:
def addToGeneDB(gene_dict, geneDB):
    """ Adds the particular sequence to the gene database
    
    Assumes the following parameters for the gene DB:
    {geneID, species, type, sequence}"""

    c.execute("INSERT INTO genes (geneID, type, species, sequence) VALUES \
                ('{geneID}', '{t}', '{sp}', '{seq}')".format(geneID = gene_dict.get("geneID"),
                                                    t = gene_dict.get("type"),
                                                    sp = gene_dict.get("organism"),
                                                    seq = gene_dict.get("sequence")))
    conn.commit()
def addToEnzymeDB(protein, geneID, enzymeDB):
    c.execute("INSERT INTO enzyme (protein, geneID) VALUES \
                ('{prot}', '{geneID}')".format(prot = protein,
                                                    geneID = geneID,
                                                    ))
    conn.commit()

In [8]:
# addToGeneDB(search('leptin', species = 'mus musculus'), "genes")

In [9]:
# search('leptin', species = 'mus musculus')

In [10]:
# Input data:
genes = {'Glycolysis': ["alcohol dehydrogenase", 
                       "hexokinase", 
                        "triosephosphate isomerase", 
                       "glucokinase"],
        'TCA': ["phosphoenolpyruvate carboxykinase",
               "pyruvate carboxylase",
               "pyruvate synthase", # K00174
               "citrate synthase"],
         'Pentose Phosphate': # [E4.3.1.9, EC 4.2.1.39   ]
             ["glucosaminate ammonia-lyase", 
               "2-dehydro-3-deoxy-phosphogluconate", 
              "gluconate dehydratase",
              "glucose oxidase"]
         
        }
species = ['Drosophila melanogaster', 'Homo sapien', 'Escherichia coli']

for key in genes:
    print("\n".join(genes[key]))

alcohol dehydrogenase
hexokinase
triosephosphate isomerase
glucokinase
phosphoenolpyruvate carboxykinase
pyruvate carboxylase
pyruvate synthase
citrate synthase
glucosaminate ammonia-lyase
2-dehydro-3-deoxy-phosphogluconate
gluconate dehydratase
glucose oxidase


In [11]:
# Helper function: visualizes the elements in the database
def sql_fetch(dbName, con = conn):
 
    cursorObj = con.cursor()
 
    cursorObj.execute('SELECT * FROM {name}'.format(name=dbName))
 
    rows = cursorObj.fetchall()
 
    for row in rows:
 
        print(row)
# sql_fetch(conn)

In [12]:
# Generate the pathway database
# def createTable(tableName, fieldsList, typesList):
#     pairs = zip(fieldsList, typesList)
#     c.execute("CREATE TABLE {t} (geneId TEXT, type TEXT, species TEXT, sequence TEXT)")
#     conn.commit()
c.execute("CREATE TABLE pathway (pathwayName TEXT, protein TEXT)")
conn.commit()

In [13]:
def addToPathwayDB(pathway, protein, geneDB):
    """ Adds the particular sequence to the pathway database
    
    Assumes the following parameters for the gene DB:
    {geneID, species, type, sequence}"""

    c.execute("INSERT INTO pathway (pathwayName, protein) VALUES \
                ('{path}', '{prot}')".format(prot = protein,
                                                    path = pathway,
                                                    ))
    conn.commit()

for pathway in genes.keys():
    for protein in genes.get(pathway):
        addToPathwayDB(pathway, protein ,"pathway")
        


In [14]:
sql_fetch("pathway")

('Glycolysis', 'alcohol dehydrogenase')
('Glycolysis', 'hexokinase')
('Glycolysis', 'triosephosphate isomerase')
('Glycolysis', 'glucokinase')
('TCA', 'phosphoenolpyruvate carboxykinase')
('TCA', 'pyruvate carboxylase')
('TCA', 'pyruvate synthase')
('TCA', 'citrate synthase')
('Pentose Phosphate', 'glucosaminate ammonia-lyase')
('Pentose Phosphate', '2-dehydro-3-deoxy-phosphogluconate')
('Pentose Phosphate', 'gluconate dehydratase')
('Pentose Phosphate', 'glucose oxidase')


In [15]:
# Now, generate the gene database via ENTREZ:
for sp in species:
    for key in genes.keys():
        for g in genes.get(key):
            result = search(g, species=sp)
            print(result)
            addToGeneDB(result, "genes")
            print(sp, g)
            time.sleep(0.3) # Adding sleep to avoid http "Too Many requests error"

{'organism': 'Drosophila melanogaster', 'geneID': 'AH000980', 'type': 'DNA', 'sequence': 'GTCGACTGCACTCGCCCCCACGAGAGAACAGTATTTAAGGAGCTGCGAAGGTCCAAGTCATGCATTATTGTCTCAGTGCAGTTGTCAGTTGCAGTTCAGCAGACGGGCTAACGAGTACTTGCATCTCTTCAAATTTACTTAATTGATCAAGTAAGTAGCAAAAGGGCACACAATTGAAGGAAATTCTTGTTTAATTGAATTTATTATGCAAGTGCGGAAATAAAATGACAGTATTAAATAGTAAATATTTTGTAAAATCATATATAATCAAATTTATTCAATCAGAACTAATTCAAGCTGTCACAAGTAGTGCGAACTCAATTAATTGGCATCGAATTAAAATTTGGAGGCCTGTTCCGCATATTCCTCTTGGAAAATCACCTGTTAGTTAACTTCTAAAAATAGGAATTTTAACATAACTCGTCCCTGTTAATCGGCGCCGTGCCTTCGTTAGCTATCTCAAAAGCGAGCGCGTGCAGACGAGCAGTAATTTTCCAAGCATCAGGCATATAATATACTAATACTAATACTAATACTAATATAAGAATACTAATATAGAAAAAGCTTTGCCGGTACAAAATCCCAAACAAAAACAAACCGTGTGTGCCGAAAAATAAAAATAAACCATAAACTAGGCAGCGCTCCGTCGCCGGCTGAGCAGCCTGCGTACATAGCCGAGATCGCGTAACGGTAGATAATGAAAAGCTCTACGTAACCGAAGCTTCTGCTGTACGGATCTTCCTATAAATACGGGGCCGACACGAACTGGAAACCAACAACTAACGGAGCCCTCTTCCAATTGAAACAGATCGAAAGAGCCTGCTAAAGCAAAAAAGAAGTCACCATGTCGTTTACTTTGACCAACAAGAACGTGATTTTCGTTGCCGGTCTGGGAGGCATTGGTCTGGAC

{'organism': 'Drosophila melanogaster', 'geneID': 'AE014296', 'type': 'DNA', 'sequence': 'N/A'}
Drosophila melanogaster glucokinase
{'organism': 'Drosophila melanogaster', 'geneID': 'NM_079060', 'type': 'mRNA', 'sequence': 'GCATCAGTTACTCGTGGCCAGAGTAAATGGAAAGATCAAGTGAAAAGCCTACTTTCAGTGGTCTGAAAATCAAGTTGATGTTTGACAAAAAACTGTGGAAGAACATATGAACGCAAAGTCCTCGACAGTAGAACACTAATTGTGATAATTACTCAAATTGTGATAAGCGAATTAAGCAAAGTGCCAAAAACCCTTTCACGCGCAGAAAAATCTAATAAAAATATTCCAAACTTTTTTTCGAGAAAAATTCATAAAAAACAACACAAACAAAATGCCTGAGCTCATTGAACAAAGCAAAATTATCTCCGGCAATGTCTGCGGTCTGCCCCAGTTGCACAAGCTGCGCCAGGACAATTGCGGTCTGTACAGCCACATCCGTGGCATTCCCATCTCCTATGGAAATGTGGATTTGCTGACCACCGGTGTCCGTGCCTTCGTGGAGGAGGGCATCGCCCTCTGCCAGCCCGACCAGGTGCACATCTGCGATGGCAGCGAGCAGGAGAACAAGGTGCTCATCAAGAGCCTCTTGGAGGCTGGCACCATTGTGCCGCTGCCCAAGTACGACAATTGCTGGCTGGCCCGCACCAATCCGGCGGATGTGGCCCGCGTCGAGTCGCGCACTTTCATCTGCACCGAGCGGCGGGAGGAGACGATTCCCACTCCAGTGGAGGGTGTCAAGGGAACCCTGGGCAACTGGATCTCGCCCAGCGACATGGATGCTGCAGTGCAGCAGCGATTCCCCGGCTGCATGAAGGGTCGCACCATGTACGTGGTGC

{'organism': 'N/A', 'geneID': 'N/A', 'type': 'N/A', 'sequence': 'N/A'}
Drosophila melanogaster glucosaminate ammonia-lyase
{'organism': 'N/A', 'geneID': 'N/A', 'type': 'N/A', 'sequence': 'N/A'}
Drosophila melanogaster 2-dehydro-3-deoxy-phosphogluconate
{'organism': 'N/A', 'geneID': 'N/A', 'type': 'N/A', 'sequence': 'N/A'}
Drosophila melanogaster gluconate dehydratase
{'organism': 'Drosophila melanogaster', 'geneID': 'NM_001363924', 'type': 'mRNA', 'sequence': 'CAGAAGCTGAGTCGGTAACGGTCTGCGTCTGCGCGCAGTTCGAACAAGTTGAGAAAGAGACCAACAGAAAGCCCATCCAAGTGCAGTGATCAATACGGGTAACTGACAAAAACCCTAGAAGTCAGGGCTTAAAAACGATTTTGCAGCGCTGCCAGTGTTTTTGTGTGATAAAAAAAAGCGGCTCAGAAAAACTTGCTGACAGCAGATAGCACACACGTTTTTGTCGTCTTCGGGCCATTGAAAAATTTTCCCGAGGCATTTTCTATAAGGAATAAACAATTAATTCAATATTTAAAGCATAAAAGGAAACTAGACACCACATCACCGGACTCTACGATCTCTATCAACATGTCCGCCAGCGCCTCAGCCTGCGATTGTTTGGTGGGCGTACCCACTGGGCCCACCCTGGCCTCCACATGTGGTGGTAGCGCCTTCATGCTGTTCATGGGCCTCCTGGAGGTCTTTATCCGCTCCCAGTGTGATCTCGAGGATCCCTGCGGAAGGGCCAGCAGTCGGTTTCGATCGGAGCCGGACT

{'organism': 'Homo sapiens', 'geneID': 'AH003112', 'type': 'DNA', 'sequence': 'N/A'}
Homo sapien glucokinase
{'organism': 'Homo sapiens', 'geneID': 'NM_002591', 'type': 'mRNA', 'sequence': 'ACGGCCTTCCCACTGGGAACACAAACTTGCTGGCGGGAAGAGCCCGGAAAGAAACCTGTGGATCTCCCTTCGAGATCATCCAAAGAGAAGAAAGGTGACCTCACATTCGTGCCCCTTAGCAGCACTCTGCAGAAATGCCTCCTCAGCTGCAAAACGGCCTGAACCTCTCGGCCAAAGTTGTCCAGGGAAGCCTGGACAGCCTACCCCAGGCAGTGAGGGAGTTTCTCGAGAATAACGCTGAGCTGTGTCAGCCTGATCACATCCACATCTGTGACGGCTCTGAGGAGGAGAATGGGCGGCTTCTGGGCCAGATGGAGGAAGAGGGCATCCTCAGGCGGCTGAAGAAGTATGACAACTGCTGGTTGGCTCTCACTGACCCCAGGGATGTGGCCAGGATCGAAAGCAAGACGGTTATCGTCACCCAAGAGCAAAGAGACACAGTGCCCATCCCCAAAACAGGCCTCAGCCAGCTCGGTCGCTGGATGTCAGAGGAGGATTTTGAGAAAGCGTTCAATGCCAGGTTCCCAGGGTGCATGAAAGGTCGCACCATGTACGTCATCCCATTCAGCATGGGGCCGCTGGGCTCGCCTCTGTCAAAGATCGGCATCGAGCTGACGGATTCACCCTACGTGGTGGCCAGCATGCGGATCATGACGCGGATGGGCACGCCCGTCCTGGAAGCAGTGGGCGATGGGGAGTTTGTCAAATGCCTCCATTCTGTGGGGTGCCCTCTGCCTTTACAAAAGCCTTTGGTCAACAACTGGCCCTGCAACCCGGAGCTGACGCTCATCGCCCACCTGCCTGACCGCA

{'organism': 'Escherichia coli MS 115-1', 'geneID': 'GG771780', 'type': 'DNA', 'sequence': 'N/A'}
Escherichia coli hexokinase
{'organism': 'Escherichia coli', 'geneID': 'EU891919', 'type': 'DNA', 'sequence': 'ATGAAAAAAGCGGGTCTTCTTTTTTTGGTGATGATAGTTATCGCCGTTGTGGCTGCCGGTATTGGTTACTGGAAATTAACCGGTGAAGAGTCGGATACATTACGTAAGATTGTCCTTGAGGAATGTTTGCCCAATCAGCAGCAAAATCAAAATCCTTCGCCATGTGCGGAAGTCAAACCCAATGCCGGATACGTGGTTTTAAAAGATCTTAATGGCCCACTGCAATATCTGTTGATGCCAACGTATCGTATTAACGGTACTGAAAGTCCTTTGTTGACCGATCCTTCAACGCCGAACTTCTTTTGGTTGGCCTGGCAGGCGCGTGATTTTATGAGCAAAAAATACGGCCAGCCGGTTCCCGATCGCGCGGTTTCTTTGGCGATCAACTCCCGCACCGGGCGTACGCAAAACCATTTTCATATTCATATCTCTTGTATTCGTCCTGATGTGCGCGAACAGCTGGATAACAATCTGGCGAACATCAGCAGCCGCTGGTTGCCACTGCCAGGTGGTTTGCGCGGGCATGAATACCTGGCGCGTCGGGTAACGGAAAGCGAACTGGTACAACGCAGCCCATTTATGATGCTGGCAGAAGAAGTACCTGAGGCGCGGGAACATATGGGAAGCTACGGGCTGGCGATGGTACGGCAGAGTGATAACTCATTTGTATTGCTGGCGACACAACGAAATCTACTGACGCTTAACCGTGCTTCAGCCGAGGAAATTCAGGATCATCAGTGTGAGATTTTGCGTTAA'}
Escherichia coli triosephosphate

In [16]:
sql_fetch("genes")

('AH000980', 'DNA', 'Drosophila melanogaster', 'GTCGACTGCACTCGCCCCCACGAGAGAACAGTATTTAAGGAGCTGCGAAGGTCCAAGTCATGCATTATTGTCTCAGTGCAGTTGTCAGTTGCAGTTCAGCAGACGGGCTAACGAGTACTTGCATCTCTTCAAATTTACTTAATTGATCAAGTAAGTAGCAAAAGGGCACACAATTGAAGGAAATTCTTGTTTAATTGAATTTATTATGCAAGTGCGGAAATAAAATGACAGTATTAAATAGTAAATATTTTGTAAAATCATATATAATCAAATTTATTCAATCAGAACTAATTCAAGCTGTCACAAGTAGTGCGAACTCAATTAATTGGCATCGAATTAAAATTTGGAGGCCTGTTCCGCATATTCCTCTTGGAAAATCACCTGTTAGTTAACTTCTAAAAATAGGAATTTTAACATAACTCGTCCCTGTTAATCGGCGCCGTGCCTTCGTTAGCTATCTCAAAAGCGAGCGCGTGCAGACGAGCAGTAATTTTCCAAGCATCAGGCATATAATATACTAATACTAATACTAATACTAATATAAGAATACTAATATAGAAAAAGCTTTGCCGGTACAAAATCCCAAACAAAAACAAACCGTGTGTGCCGAAAAATAAAAATAAACCATAAACTAGGCAGCGCTCCGTCGCCGGCTGAGCAGCCTGCGTACATAGCCGAGATCGCGTAACGGTAGATAATGAAAAGCTCTACGTAACCGAAGCTTCTGCTGTACGGATCTTCCTATAAATACGGGGCCGACACGAACTGGAAACCAACAACTAACGGAGCCCTCTTCCAATTGAAACAGATCGAAAGAGCCTGCTAAAGCAAAAAAGAAGTCACCATGTCGTTTACTTTGACCAACAAGAACGTGATTTTCGTTGCCGGTCTGGGAGGCATTGGTCTGGACACCAGCAAGGAGCTGCTCAAGCGCGATCTGAAGGTAACAATG